# 文本解析与内容提取

## 利用Beautiful Soup处理HTML文档

Beautiful Soup不是python标准库，所以需要下载安装后才能使用。在anaconda命令行运行：

``` conda install beautifulsoup4``` 

或者

``` conda install bs4```

In [1]:
# 如果这个语句能正常运行，则bs4安装正确
from bs4 import BeautifulSoup

In [2]:
html = '''
<!DOCTYPE html>
<html lang="zh">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <title>title of the sample</title>
    <link rel="stylesheet" type="text/css" href="mystyle.css" />
    <script type="text/javascript" src="http://no.where/js2.js" />
</head>
<body>
<h1  style="font-family:verdana"> heading one </h1>   <hr />
<a class="link" href="http://www.w3school.com.cn/" id="link1">Visit W3School</a>
<p class="poem"> 天生我材必有用 </p>
<a class="link" href="http://www.ustc.edu.cn" id="link2">USTC, where dreams come true</a>
<p class="text">This is<br />a para<br />graph with line breaks</p>
<p class="poem"> 白云生处有人家 </p>
<ol><li>item 1</li></ol>
<table border="1">
<tr> <td>row 1, cell 1</td> <td>row 1, cell 2</td> </tr>
<tr> <td>row 2, cell 1</td> <td>row 2, cell 2</td> </tr>
</table>
</body></html>
'''

### 工具试用

1. 提取网页标题；

2. 提取网页中的锚（anchor）链接(href)及其文本；

3. 提取网页中class标记为poem的段落（P）文本。

In [3]:
# 创建一个bs4.Beautifulsoup对象，指定使用python的标准html解析器
doc1 = BeautifulSoup(html, 'html.parser')

In [4]:
print('the title of the web page: ', doc1.title.get_text())
for anchor in doc1.find_all('a'):
    print(anchor.get_text(), '->', anchor.get('href'))
for poem in doc1.select('.poem'):
    print(poem.get_text())

the title of the web page:  title of the sample
Visit W3School -> http://www.w3school.com.cn/
USTC, where dreams come true -> http://www.ustc.edu.cn
 天生我材必有用 
 白云生处有人家 


In [5]:
type(doc1), type(doc1.a), type(doc1.a.attrs)

(bs4.BeautifulSoup, bs4.element.Tag, dict)

In [6]:
print("tag object:", doc1.a)
print("attributes: ", doc1.a.attrs)
print("attribute: class = ", doc1.a.attrs['class'])
print("text of the tag: ", doc1.a.string)

tag object: <a class="link" href="http://www.w3school.com.cn/" id="link1">Visit W3School</a>
attributes:  {'class': ['link'], 'href': 'http://www.w3school.com.cn/', 'id': 'link1'}
attribute: class =  ['link']
text of the tag:  Visit W3School


### Beautifulsoup的解析

Beautifulsoup将HTML文档解析为一棵文档树，每个节点解释为对象。

既然是树结构，就牵涉到树的层级，如父子关系，兄弟关系。可以使用prettify()直观查看树的层级结构。

标签的名称、属性和标签内的文本均可以使用标签对象的属性和方法来访问。

In [7]:
#print(doc1.prettify())  # the document tree

### Beautifulsoup的对象类型

* Tag对象，对应于HTML/XML标签
* NavigableString对象，对应于标签内的文本
* Beautifulsoup对象，表示文档对象
* Comment对象，对应于注释

In [8]:
type(doc1.title), type(doc1), type(doc1.title.string)

(bs4.element.Tag, bs4.BeautifulSoup, bs4.element.NavigableString)

In [9]:
#doc1.__dict__
doc1.title.__dict__
#doc1.title.string.__dict__

{'attrs': {},
 'can_be_empty_element': False,
 'contents': ['title of the sample'],
 'hidden': False,
 'known_xml': False,
 'name': 'title',
 'namespace': None,
 'next_element': 'title of the sample',
 'next_sibling': '\n',
 'parent': <head>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <title>title of the sample</title>
 <link href="mystyle.css" rel="stylesheet" type="text/css"/>
 <script src="http://no.where/js2.js" type="text/javascript"></script>
 </head>,
 'parser_class': bs4.BeautifulSoup,
 'prefix': None,
 'preserve_whitespace_tags': {'pre', 'textarea'},
 'previous_element': '\n',
 'previous_sibling': '\n'}

由于文档节点在文档树中，所以节点对象基本都有children, parents, previous_element, next_element, previous_sibling, next_sibling等属性，用来定位其在文档树中的位置。

Tag对象和Beautifulsoup对象都有属性name和attrs，帮助我们访问标签的特征。

In [10]:
print(doc1.a.name)
print(doc1.a.attrs)
print(doc1.name)
print(doc1.attrs)

a
{'class': ['link'], 'href': 'http://www.w3school.com.cn/', 'id': 'link1'}
[document]
{}


### Tag对象的属性和方法

+ 标签对象的属性

  * 标签的名称
  * 标签的属性和属性值
  * 标签内的文本

In [11]:
print(doc1.a.name)
print(doc1.a.attrs)
print(doc1.a.attrs['href'])

a
{'class': ['link'], 'href': 'http://www.w3school.com.cn/', 'id': 'link1'}
http://www.w3school.com.cn/


除了使用标签的attrs字典来访问属性和值之外，我们也可以直接对标签对象使用下标来访问，或者使用get方法获取其属性值。

In [12]:
print(doc1.a['href'])
print(doc1.a.get('href'))  # 更安全，原因？

http://www.w3school.com.cn/
http://www.w3school.com.cn/


若需要获取标签内的文本，则需要使用string属性，或text属性，或get_text方法。

+ string与text属性的区别：

  * string属性，获得当前标签的文本，若当前标签包含多个标签则返回None；
  * text属性，当前标签及其子节点内的所有文本（拼接）形成的字符串
  
+ strings属性
  * 当前标签内所有文本的生成器，用于遍历

In [13]:
print(doc1.a.string)
print(doc1.a.text)
print(doc1.a.get_text())

Visit W3School
Visit W3School
Visit W3School


In [14]:
print(doc1.tr.string)  #  当前标签内子标签不止一个
print(doc1.ol.string)  #  特殊情况，标签内只有一个子标签，返回其内层对象的string
print(doc1.tr.text)
print(doc1.tr.get_text())

None
item 1
 row 1, cell 1 row 1, cell 2 
 row 1, cell 1 row 1, cell 2 


+ 获取Tag对象的上下文信息，可以访问下列属性；

  * contents
  * children
  * descendants
  * parent
  * parents

In [15]:
doc1.head.contents  # 注意这不是一个字符串的列表
#type(doc1.head.contents[1])

['\n',
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 '\n',
 <title>title of the sample</title>,
 '\n',
 <link href="mystyle.css" rel="stylesheet" type="text/css"/>,
 '\n',
 <script src="http://no.where/js2.js" type="text/javascript"></script>,
 '\n']

In [16]:
for i in doc1.table.children:  # 遍历直接子节点
    if i.name: print(i.name)

tr
tr


In [17]:
for i in doc1.table.descendants:  # 递归遍历子孙节点
    if i.name: print(i.name)

tr
td
td
tr
td
td


In [18]:
print(type(doc1.head.contents),type(doc1.head.children),type(doc1.head.descendants))
print(type(doc1.head.parent),type(doc1.head.parents))

<class 'list'> <class 'list_iterator'> <class 'generator'>
<class 'bs4.element.Tag'> <class 'generator'>


+ 上下文访问小结：

  * contents, 获取直接子节点的列表，其中部分节点可能为回车符构成的NavigableString
  * children，获取子节点的迭代器
  * descendants， 递归获取所有子孙节点的生成器
  * parent，获取父节点对象
  * parents，获取父节点的生成器

### 文档树的遍历



In [19]:
def walk_over(node, level):
    for i in node.children:
        if i.name:  # 如果不是标签节点，则名称为None
            print('-' * level, i.name)
            walk_over(i, level = level + 1)

In [20]:
level = 0
walk_over(doc1.body, level)

 h1
 hr
 a
 p
 a
 p
- br
- br
 p
 ol
- li
 table
- tr
-- td
-- td
- tr
-- td
-- td


+ 当前节点的遍历

  * 遍历当前节点的子节点，使用 contents 或者 children
  * 遍历当前节点的所有子孙节点，使用 descendants


In [21]:
for i in doc1.head.children:
    if i.name:  print(i.name)

meta
title
link
script


+ 文档树的遍历

  * 遍历之前/之后的兄弟节点
  * 遍历之前/之后的节点
  * 向上遍历

In [22]:
for i in doc1.h1.next_siblings:
    if i.name:  print(i.name)

hr
a
p
a
p
p
ol
table


In [23]:
for i in doc1.ol.next_elements:
    if i.name:  print(i.name)

li
table
tr
td
td
tr
td
td


* next_sibling 下一个兄弟节点
* next_siblings 下个兄弟节点迭代器
* next_element 下一个节点，不分层次
* next_elements 下个节点迭代器

前缀换成previous指的是上一个，sibling和element等含义类似

In [24]:
for i in doc1.a.parents:
    if i.name:  print(i.name)

body
html
[document]


向上遍历

* parent， 当前节点的父节点
* parents， 父节点迭代器

## 文档树的搜索

* Beautifulsoup.find_all方法，作用：提取满足要求的标签对象列表。

find_all方法的第一个参数为name，指的是标签名

In [25]:
doc1.find_all('p')

[<p class="poem"> 天生我材必有用 </p>,
 <p class="text">This is<br/>a para<br/>graph with line breaks</p>,
 <p class="poem"> 白云生处有人家 </p>]

In [26]:
doc1.find_all(['h1','td'])

[<h1 style="font-family:verdana"> heading one </h1>,
 <td>row 1, cell 1</td>,
 <td>row 1, cell 2</td>,
 <td>row 2, cell 1</td>,
 <td>row 2, cell 2</td>]

+ name参数的几种情形

  * 字符串，匹配标签名
  * 字符串列表，匹配多个标签
  * True，找出所有标签
  * 正则表达式，匹配所有满足表达式的标签
  * 函数，返回True的Tag

In [27]:
#例如：在文档中找到所有标题类标签。
import re
doc1.find_all(re.compile("^h\d"))

[<h1 style="font-family:verdana"> heading one </h1>]

find_all方法的第二个参数为attrs，指的是标签的属性，需要指定属性名和值的键值对进行筛选。

In [28]:
doc1.find_all(attrs = {'class':'poem'})

[<p class="poem"> 天生我材必有用 </p>, <p class="poem"> 白云生处有人家 </p>]

In [29]:
doc1.find_all(attrs = {'class':'link'})

[<a class="link" href="http://www.w3school.com.cn/" id="link1">Visit W3School</a>,
 <a class="link" href="http://www.ustc.edu.cn" id="link2">USTC, where dreams come true</a>]

若需要搜索标签的文本，则可以指定参数text.

In [30]:
doc1.find_all(text = re.compile('白云'))

[' 白云生处有人家 ']

* Beautifulsoup.find_all小结

```
  Beautifulsoup.find_all(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs)
```
name = 标签的名称或匹配准则，attrs=由(属性名=匹配准则)构成的字典，对属性的值进行过滤（准则可以是字符串/字符串列表/正则表达式等）。

recursive =是否递归检索所有子孙节点；text=搜索指定文本；limit=限制搜索结果量

## 使用CSS选择器提取内容

常用CSS选择器

* 标签选择器，对于同一种HTML标签指定样式；

* 类别选择器(class)，对于同一个类别指定样式；

* ID选择器，对于特定元素指定样式，ID是唯一的。

CSS示例，假设我们有一个style.css文件，样式表如下：

```
  p{font-size:12px; background:#0000CC;}

  .peom{color:#FF0000; }

  #link1{color:#FF0000;}

```
分别指定了P（段落）标签的样式，类别(class=)peom的样式，ID(id=)link1的样式

利用CSS选择器提取数据时，我们可以使用soup对象的select方法。

* 利用标签提取

In [31]:
p = doc1.select('p')
p

[<p class="poem"> 天生我材必有用 </p>,
 <p class="text">This is<br/>a para<br/>graph with line breaks</p>,
 <p class="poem"> 白云生处有人家 </p>]

In [32]:
print('data type: ', type(p))
print('data length: ', len(p))
for i in p:
    print(i)

data type:  <class 'list'>
data length:  3
<p class="poem"> 天生我材必有用 </p>
<p class="text">This is<br/>a para<br/>graph with line breaks</p>
<p class="poem"> 白云生处有人家 </p>


In [33]:
for i in p:
    print(i.get_text())

 天生我材必有用 
This isa paragraph with line breaks
 白云生处有人家 


In [34]:
doc1.select('a')

[<a class="link" href="http://www.w3school.com.cn/" id="link1">Visit W3School</a>,
 <a class="link" href="http://www.ustc.edu.cn" id="link2">USTC, where dreams come true</a>]

In [35]:
for i in doc1.select('a'):
    print(i.get('class'), i.get('href'), i.string)

['link'] http://www.w3school.com.cn/ Visit W3School
['link'] http://www.ustc.edu.cn USTC, where dreams come true


In [36]:
for td in doc1.select('td'):
    print(td.text)

row 1, cell 1
row 1, cell 2
row 2, cell 1
row 2, cell 2


In [37]:
for tr in doc1.select('tr'):
    print(tr.text)  # 注意和tr.string的结果进行区分

 row 1, cell 1 row 1, cell 2 
 row 2, cell 1 row 2, cell 2 


* 利用类别(class)提取


In [38]:
print(doc1.select('.poem'))
print(doc1.select('link'))

[<p class="poem"> 天生我材必有用 </p>, <p class="poem"> 白云生处有人家 </p>]
[<link href="mystyle.css" rel="stylesheet" type="text/css"/>]


* 利用ID提取

注意ID虽然要求是唯一的，但是select仍然返回列表

In [39]:
doc1.select('#link1')

[<a class="link" href="http://www.w3school.com.cn/" id="link1">Visit W3School</a>]

* CSS选择器小结
  - 选择标签时，标签名不需要修饰；

  - 选择类别时，需要前缀句点'.'

  - 选择ID时，需要前缀'#'
  
  - select方法总是返回一个列表

In [40]:
# 组合选择
print(doc1.select('p.poem'))
print(doc1.select('p[class="poem"]'))

[<p class="poem"> 天生我材必有用 </p>, <p class="poem"> 白云生处有人家 </p>]
[<p class="poem"> 天生我材必有用 </p>, <p class="poem"> 白云生处有人家 </p>]


## 解析器选项

--------
| 解析器  |  参数 |
|----|----|
| 内置HTML解析器  |  "html.parser" |
| lxml HTML解析器  |  "lxml" |
| lxml XML解析器  |  "xml" |
| html5lib  |  "html5lib" |


推荐使用lxml作为解析器,因为它的解析速度快，容错能力也比较强。

如果一段HTML或XML文档格式不正确的话,那么在不同的解析器中返回的结果可能是不一样的。

## 从文件/网络进行解析

拷贝一个html文件到当前目录，假设文件名为sample.html。

Python会帮我们完成文件解码的问题，并将所有字符串转换为unicode字符串。

下面我们从互联网下载一个网页进行解析。

In [41]:
#doc2 = BeautifulSoup(open("sample.html"))

In [42]:
import urllib.request
f = urllib.request.urlopen("http://news.baidu.com")

In [43]:
source = f.read().decode()
doc2 = BeautifulSoup(source, 'lxml')

In [44]:
doc2.find_all('a')[:5]

[<a data-path="s?wd=" href="https://www.baidu.com/">网页</a>,
 <a data-path="f?kw=" href="http://tieba.baidu.com/">贴吧</a>,
 <a data-path="search?ct=17&amp;pn=0&amp;tn=ikaslist&amp;rn=10&amp;lm=0&amp;word=" href="https://zhidao.baidu.com/">知道</a>,
 <a data-path="search?fr=news&amp;ie=utf-8&amp;key=" href="http://music.baidu.com/">音乐</a>,
 <a data-path="search/index?ct=201326592&amp;cl=2&amp;lm=-1&amp;tn=baiduimage&amp;istype=2&amp;fm=&amp;pv=&amp;z=0&amp;word=" href="http://image.baidu.com/">图片</a>]

官方参考：

https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/
